In [ ]:
import json
import pathlib
import sys

import matplotlib.pyplot as plt
import numpy as np
import pydicom as pdc
import tensorflow as tf

sys.path.append(pathlib.Path.cwd().parent.as_posix())
from src.data.preprocess.lib.tfrecord import create_example_fn, parsed_example_fn
from src.data.preprocess.lib.utils import get_pos_from_bin_list, get_pos_from_mult_list
from src.models.lib.data_loader import preprocess_img

In [ ]:
project_root_path = pathlib.Path.cwd().parent

patient_idx = 5
patient_img_idx = 1

clean_json_path = list(project_root_path.rglob("clean*.json"))[0]

with clean_json_path.open(mode="r") as json_file:
    clean_dict_output = json.load(json_file)

patient_info = clean_dict_output[str(patient_idx).zfill(3)]


patient_img_info = patient_info[patient_img_idx]

patient_img_num = patient_img_info["idx"]

print(
    f"Patient {patient_idx} Image {patient_img_idx+1}/{len(patient_info)} ({patient_img_num})"
)

patient_root_path = next(project_root_path.rglob(f"patient/{patient_idx}"))
patient_dcm_path = next(
    patient_root_path.rglob(f"*00{str(patient_img_num).lstrip('0').zfill(2)}.dcm")
)

patient_dcm = pdc.dcmread(patient_dcm_path)
patient_img_arr = patient_dcm.pixel_array
patient_img_hu = pdc.pixel_data_handlers.util.apply_modality_lut(
    patient_img_arr, patient_dcm
)

patient_roi = patient_img_info["roi"]
plot_dict = {}

for roi in patient_roi:
    vessel_loc = roi["loc"]
    vessel_pos = plot_dict.get(vessel_loc, [])
    vessel_pos.extend(roi["pos"])
    plot_dict[vessel_loc] = vessel_pos

plot_dictfig, ax = plt.subplots(1, 3)

ax[0].set_title("Image")
ax[0].imshow(patient_img_arr, cmap="gray", interpolation="none")

ax[1].set_title("Multiclass Segment")
ax[1].imshow(np.ones([512, 512]), cmap="gray")

for loc, p in plot_dict.items():
    temp = np.zeros((512, 512))
    temp[tuple(zip(*p))] = 1
    ax[1].imshow(np.transpose(temp), cmap="gray", alpha=0.4)

ax[2].set_title("ROI Overlay")
ax[2].imshow(patient_img_arr, cmap="gray", interpolation="none")
for loc, p in plot_dict.items():
    temp = np.zeros((512, 512))
    temp[tuple(zip(*p))] = 1
    ax[2].imshow(np.transpose(temp), cmap="gray", alpha=0.5)
    print(loc)
    # break

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.show()